# Mapping Hurricane Harvey

## Set Up and Initialization

We will be start by using three modules in this program: ee, folium, and geehydro.  
1) ee allows us to interface with Google Earth Engine in a Python development environment. Through GEE, we can aquire publically available satellite data from Sentinel-I.  
2) folium is used to display map data. It will generate a html file at the very end of our development which can be opened and shared.  
3) geehydro adds some additional functionality to folium. Namely, some of the functions available on GEE's Javascript Code Editor are not available just by importing ee and folium.  

In [1]:
import ee, folium, geehydro
import pandas as pd

ee.Authenticate() # You only need to run this command once. It authenticates your EE Developer credentials.
ee.Initialize()

## Data Collection

In [ ]:
# Houston's coordinates stored in a Point construct
houston_point = ee.Geometry.Point([-95.37, 29.76])

# 8/1 to 8/16 is our before; 8/17 to 9/2 is our after
harvey_collect = sentinel1\
.filter(ee.Filter.date('2017-08-01', '2017-09-02'))\ # starts one week before Harvey
.filterBounds(houston_point)\ # geographical filtering
.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\ 
.select('VV') # VV bands work best with speckle filtering, so we'll isolate those

# Filter by date {before, after}: create mosaics out of overlapping Sentinel imagery
    # before provides a baseline for how Houston "normally" looks
    # after shows us how Houston looked after Harvey hit
before = harvey_collect.filterDate('2017-08-01', '2017-08-16').mosaic() 
after = harvey_collect.filterDate('2017-08-17', '2017-09-02').mosaic()

## Data Processing

In [ ]:
# Here, we perform speckle filtering (can also be done on SNAP, 
#     the ESA's desktop program for working with Sentinel imagery).
SMOOTHING_RADIUS = 100
DIFF_UPPER_THRESHOLD = -3
diff_smoothed = after.focal_median(SMOOTHING_RADIUS, 'circle', 'meters') \
    .subtract(before.focal_median(SMOOTHING_RADIUS, 'circle', 'meters'))
diff_thresholded = diff_smoothed.lt(DIFF_UPPER_THRESHOLD)

## Map Display Pt. 1 - Flooding

In [ ]:
Map = folium.Map()
Map.centerObject(houston_point, 13)
Map.addLayer(diff_thresholded.updateMask(diff_thresholded), {'palette': '0000FF'}, 'flooded areas - blue')

## Map Display Pt. 2 - High Water Marks

We'll be reading in the High Water Mark data from a CSV file (linked in the readme). To get data from a CSV, we need one more module: Pandas.

In [ ]:
import pandas as pd

# Replace this with the path/name of the CSV file on your system.
FILE_NAME = './Harvey_HWMs.csv'
hwm = pd.read_csv(FILE_NAME)
# There were thousands of HWMs recorded during Harvey. 
#    To make the map more understandable, we introduce a minimum height above ground.
HWM_THRESHOLD = 0
for i in range(hwm['height_above_gnd'].size):
    if (hwm['height_above_gnd'][i] > HWM_THRESHOLD):
        folium.Marker(location = [hwm['latitude'][i], hwm['longitude'][i]]).add_to(Map)

## Saving and Displaying the Map

In [ ]:
MAP_NAME = 'HoustonHarvey_HWM' # Change this to whatever file name you wish.
# The map will be saved as an HTML file which displays an OpenStreetMap when clicked.
Map.save('HoustonHarvey_HighWaterMarks.html') 